#### Exercise 7.1

`Par.map2` is a new higher-order function for combining the result of two parallel computations. What is its signature?

```scala
def map2[A, B, C](a: Par[A])(b: Par[B])(f: (A, B) => C): Par[C]
```

#### Exercise 7.2

Try to come up with the representation for `Par` that make it possible to implement the functions of our API.

In [1]:
import java.util.concurrent.{ExecutorService, Callable, Future, TimeUnit}

import java.util.concurrent.{ExecutorService, Callable, Future, TimeUnit}


In [2]:
type Par[A] = ExecutorService => Future[A]

extension [A](pa: Par[A])
    def run(s: ExecutorService): Future[A] = pa(s)

defined type Par
defined extension methods 

In [8]:
object Par:
    def unit[A](a: A): Par[A] = es => UnitFuture(a)
    def lazyUnit[A](a: => A): Par[A] = fork(unit(a))
    def fork[A](a: => Par[A]): Par[A] =
        es => es.submit(() => a(es).get)

case class UnitFuture[A](get: A) extends Future[A]:
    def isDone = true
    def get(timeout: Long, units: TimeUnit) = get
    def isCancelled = false
    def cancel(evenIfRunning: Boolean): Boolean = false

extension [A](pa: Par[A])
    def map2[B, C](pb: Par[B])(f: (A, B) => C): Par[C] =
        (es: ExecutorService) =>
            val futureA = pa(es)
            val futureB = pb(es)
            UnitFuture(f(futureA.get, futureB.get))


defined object Par
defined class UnitFuture
defined extension methods 

#### Exercise 7.3

*Hard*: Fix the implementaiton of `map2` so it respects the contract of timeouts on `Future`.

In [9]:
case class Map2Future[A, B, C](futureA: Future[A], futureB: Future[B], f: (A, B) => C) extends Future[C]:
    def isDone = true
    def isCancelled = false
    def get = f(futureA.get, futureB.get)
    def cancel(evenIfRunning: Boolean): Boolean = false
    def get(timeout: Long, units: TimeUnit) =
        val timeoutNanos = units.toNanos(timeout)
        val endNanos = System.nanoTime() + timeoutNanos

        val a = futureA.get(timeoutNanos, TimeUnit.NANOSECONDS)

        val timeleft = endNanos - System.nanoTime()
        val b = futureB.get(timeleft, TimeUnit.NANOSECONDS)

        f(a, b)


defined class Map2Future

#### Exercise 7.4

This API already enables a rich set of operations. Here's a simple example. Using `lazyUnit`, write a function to convert any function `A => B` to one that evaluates its result asynchronously.

In [11]:
def asyncF[A, B](f: A => B): A => Par[B] =
    a => Par.lazyUnit(f(a))

defined function asyncF

#### Exercise 7.5

Write this function, called `sequence`. No additional primitivies are required; do not call `run`.

In [12]:
def sequence[A](ps: List[Par[A]]): Par[List[A]] =
    ps.foldRight(Par.unit(Nil:List[A]))((p, acc) => p.map2(acc)(_ :: _))

defined function sequence

In [14]:
def parMap[A, B](ps: List[A])(f: A => B): Par[List[B]] =
    Par.fork {
        val fbs : List[Par[B]] = ps.map(asyncF(f))
        sequence(fbs)
    }

defined function parMap

#### Exercise 7.6

Implement `parFilter`, which filters elements of a list in parallel

In [17]:
def parFilter[A](as: List[A])(f: A => Boolean): Par[List[A]] =
    Par.fork {
        val pbools = parMap(as)(f)
        Par.unit(as).map2(pbools)((as, bs) => 
            as.zip(bs).filter(_._2).map(_._1)
        )
    }

defined function parFilter